# Effect of repealing Child Tax Credit with Child Dividend

This identifies beneficiaries of the Child Tax Credit by modeling its repeal. Both repeal from current (2017) state and TCJA state are considered on a static basis. Change to after-tax income by decile and share of after-tax income held by top 10% are calculated.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-02-20*

## Setup

### Imports

In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [3]:
tc.__version__

'0.16.1'

In [4]:
sns.set_style('white')
DPI = 300
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI

### Create policies

Load reforms from GitHub. Policies and reforms are named according to this convention:

**`{reform}_{year}_{object}`**

For example:

* `noctc_2017_policy`: Policy without CTC using 2017 law.
* `ubi_2018_calc`: Calculator replacing CTC with UBI using 2018 (current) law.

In [5]:
# Folders where reforms live.
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/'

TAXCALC_GITHUB_BASE_URL = (GITHUB_BASE_URL +
                           'open-source-economics/Tax-Calculator/master/' +
                           'taxcalc/reforms/')

def read_url(url):
    return url_lib.urlopen(url).read()

def read_reform_taxcalc_github(reform_name):
    return read_url(TAXCALC_GITHUB_BASE_URL + reform_name + '.json')

def policy_from_reform(reform):
    pol = tc.Policy()
    pol.implement_reform(reform['policy'])
    if pol.reform_errors:
        print(pol.reform_errors)
    return pol

def create_static_policy_taxcalc_github(reform_name):
    reform = tc.Calculator.read_json_param_objects(
        read_reform_taxcalc_github(reform_name), None)
    return policy_from_reform(reform)

In [6]:
y2017_policy = create_static_policy_taxcalc_github(
    '2017_law')

### Specify `Calculator` objects for static analyses

In [8]:
recs = tc.Records.cps_constructor()

In [9]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [10]:
def weighted_sum(df, col):
    return (df[col] * df['s006']).sum()

In [11]:
def child_ubi_reform(amount):
    return {2018: {'_UBI_u18': [amount],
                   '_UBI_ecrt': [1.0]}}

In [12]:
np.where(False, 1, np.nan)

array(nan)

In [13]:
def add_weighted_quantiles(df, col):
    df.sort_values(by=col, inplace=True)
    col_pctile = col + '_percentile_exact'
    df[col_pctile] = 100 * df['s006'].cumsum() / df['s006'].sum()
    # "Null out" negatives using -1, since integer arrays can't be NaN.
    # TODO: Should these be null floats?
    df[col_pctile] = np.where(df[col] >= 0, df[col_pctile], 0)
    # Reduce top record, otherwise it's incorrectly rounded up.
    df[col_pctile] = np.where(df[col_pctile] >= 99.99999, 99.99999, df[col_pctile])
    df[col + '_percentile'] = np.ceil(df[col_pctile]).astype(int)
    df[col + '_decile'] = np.ceil(df[col_pctile] / 10).astype(int)
    df[col + '_quintile'] = np.ceil(df[col_pctile] / 20).astype(int)
    return df

In [14]:
def static_calc(use_2017_law=False, 
                ctc_treatment='keep',
                child_ubi_amount=0,
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income', 'nu18', 'n24', 'XTOT']):
    """Creates static Calculator.

    Args:
        use_2017_law: Whether to use 2017 law vs. current law. Defaults to False.
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'rev_neutral_ubi': Replace with revenue-neutral child UBI.
            * 'no_cut_ubi': [NOT YET IMPLEMENTED] 
                Replace with a child UBI equal to the current maximum value.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18', 'n24', 'XTOT'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of after-tax income.
    """
    # Initiate policy using either 2017 or current law.
    if use_2017_law:
        pol = copy.deepcopy(y2017_policy)
    else:
        pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment != 'keep':
        pol.implement_reform(noctc_reform)
    if child_ubi_amount > 0:
        pol.implement_reform(child_ubi_reform(child_ubi_amount))
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol)
    calc.advance_to_year(year)
    calc.calc_all()
    # TODO: Calculate revenue for revenue-neutral UBI.
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    # Add percentiles.
    df = add_weighted_quantiles(df, 'expanded_income')
    df = add_weighted_quantiles(df, 'aftertax_income')
    # Add identifiers.
    df['use_2017_law'] = use_2017_law
    df['ctc_treatment'] = ctc_treatment
    # What's the column for the ID?
    df['id'] = df.index
    return df

In [15]:
scenarios_pre_ubi = pd.concat([
    static_calc(use_2017_law=False, ctc_treatment='keep'),
    static_calc(use_2017_law=False, ctc_treatment='repeal'),
    static_calc(use_2017_law=True, ctc_treatment='keep'),
    static_calc(use_2017_law=True, ctc_treatment='repeal')])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [16]:
scenarios_pre_ubi.head()

,s006,aftertax_income,expanded_income,nu18,n24,XTOT,expanded_income_percentile_exact,expanded_income_percentile,expanded_income_decile,expanded_income_quintile,aftertax_income_percentile_exact,aftertax_income_percentile,aftertax_income_decile,aftertax_income_quintile,use_2017_law,ctc_treatment,id
37355,60.49,-3.555567e+06,-3.542510e+06,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,37355
394742,499.17,-9.314900e+05,-9.311799e+05,1.0,1.0,3.0,0.0,0,0,0,0.0,0,0,0,False,keep,394742
29539,1209.73,-8.047257e+05,-8.047257e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,29539
276823,476.72,-7.817199e+05,-7.817199e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,276823
306673,399.35,-7.776941e+05,-7.753810e+05,0.0,0.0,2.0,0.0,0,0,0,0.0,0,0,0,False,keep,306673


Calculate child allowance.

In [17]:
aftertax_income_summary = (
    scenarios_pre_ubi.groupby(['use_2017_law', 'ctc_treatment'], as_index=False).
    apply(lambda x: 
          pd.Series({
              'aftertax_income': weighted_sum(x, 'aftertax_income'), 
              'nu18': weighted_sum(x, 'nu18')}))).reset_index()
nu18_total = aftertax_income_summary['nu18'][0]

In [18]:
aftertax_income_chg = aftertax_income_summary.pivot(
    index='use_2017_law',
    columns='ctc_treatment',
    values='aftertax_income'
)
aftertax_income_chg['chg'] = aftertax_income_chg['keep'] - aftertax_income_chg['repeal']
aftertax_income_chg['child_ubi'] = aftertax_income_chg['chg'] / nu18_total
aftertax_income_chg

ctc_treatment,keep,repeal,chg,child_ubi
use_2017_law,,,,
False,1.095007e+13,1.082995e+13,1.201240e+11,1464.524234
True,1.081362e+13,1.076178e+13,5.184284e+10,632.056117


In [19]:
scenarios = pd.concat([
    scenarios_pre_ubi,
    static_calc(use_2017_law=False, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[False, 'child_ubi']),
    static_calc(use_2017_law=True, ctc_treatment='rev_neutral_ubi',
                child_ubi_amount=aftertax_income_chg.loc[True, 'child_ubi'])
])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Analysis

* Add percentile to previous steps.
* Should there be a field for more "gross" income? Without benefits and UBI.

### Differences between `nu18` and `n24`

In [20]:
keep_cur = scenarios[(~scenarios['use_2017_law']) & 
                     (scenarios['ctc_treatment'] == 'keep')]
print ('Total children under 18: ' +
       '{:0.1f}M'.format((keep_cur['nu18'] * keep_cur['s006']).sum() / 1e6))
print ('Total children eligible for CTC: ' +
       '{:0.1f}M'.format((keep_cur['n24'] * keep_cur['s006']).sum() / 1e6))

Total children under 18: 82.0M
Total children eligible for CTC: 81.9M


How many households have nu18 > n24? n24 > nu18?

In [21]:
keep_cur.pivot_table(index='n24', columns='nu18', values='s006', aggfunc=sum)

nu18,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
n24,,,,,,,,,,,,,
0.0,1.149045e+08,5.766988e+06,547510.00,211161.64,69313.39,18471.38,5404.80,2750.33,50.61,3008.35,NaN,NaN,NaN
1.0,4.757063e+06,1.655386e+07,1788753.63,87158.45,16146.37,3154.00,218.94,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2.244122e+06,8.548261e+05,12673415.39,928240.83,47172.02,7316.58,1759.92,486.02,NaN,NaN,NaN,NaN,NaN
3.0,1.029664e+06,4.764278e+05,28834.94,4614543.24,1584990.14,459044.89,135175.92,47639.99,15654.63,5527.24,4263.23,118.96,147.68


In [22]:
keep_cur[keep_cur['nu18'] > keep_cur['n24']]['s006'].sum() / 1e6

11.757627969999998

In [23]:
keep_cur[keep_cur['n24'] > keep_cur['nu18']]['s006'].sum() / 1e6

9.3909378300000004

### Distributional impact

In [24]:
scenario_pivot = scenarios.pivot_table(values=['expanded_income', 'aftertax_income'],
                                       index=['id', 'use_2017_law', 's006'],
                                       columns='ctc_treatment').reset_index()
# Adapted from https://stackoverflow.com/q/42099024/1840471.
scenario_pivot.columns = ["_".join((j, i)) for i, j in scenario_pivot.columns]
scenario_pivot.columns = scenario_pivot.columns.str.lstrip('_')

In [25]:
scenario_pivot['afti_chg_ctc'] = (
    (scenario_pivot['repeal_aftertax_income'] - scenario_pivot['keep_aftertax_income']) / 
    scenario_pivot['keep_aftertax_income'])
scenario_pivot['afti_chg_ubi'] = (
    (scenario_pivot['rev_neutral_ubi_aftertax_income'] - scenario_pivot['keep_aftertax_income']) / 
    scenario_pivot['keep_aftertax_income'])
scenario18 = scenario_pivot[~scenario_pivot['use_2017_law']].drop(columns='use_2017_law')

In [26]:
scenario18[(scenario18['rev_neutral_ubi_aftertax_income'] >= 0) &
           (scenario18['keep_aftertax_income'] >= 0) &
           (scenario18['afti_chg_ubi'] < np.inf)].sort_values('afti_chg_ubi').tail()

,id,s006,keep_aftertax_income,repeal_aftertax_income,rev_neutral_ubi_aftertax_income,keep_expanded_income,repeal_expanded_income,rev_neutral_ubi_expanded_income,afti_chg_ctc,afti_chg_ubi
321318,160659,824.31,1.739164,1.739164,2930.787633,1.739164,1.739164,2930.787633,0.0,1684.170422
318758,159379,457.08,1.739164,1.739164,2930.787633,1.739164,1.739164,2930.787633,0.0,1684.170422
413628,206814,1007.05,1.549691,1.549691,4395.122394,1.549691,1.549691,4395.122394,0.0,2835.128587
258024,129012,203.45,1.541713,1.541713,4395.114416,1.541713,1.541713,4395.114416,0.0,2849.799369
450262,225131,355.67,1.363286,1.363286,4394.935990,1.363286,1.363286,4394.935990,0.0,3222.781213


Merge to baseline after-tax income quantiles.

In [27]:
scenarios.columns

Index([u's006', u'aftertax_income', u'expanded_income', u'nu18', u'n24',
       u'XTOT', u'expanded_income_percentile_exact',
       u'expanded_income_percentile', u'expanded_income_decile',
       u'expanded_income_quintile', u'aftertax_income_percentile_exact',
       u'aftertax_income_percentile', u'aftertax_income_decile',
       u'aftertax_income_quintile', u'use_2017_law', u'ctc_treatment', u'id'],
      dtype='object')

In [28]:
base_aftiq18 = scenarios.loc[(~scenarios['use_2017_law']) & (scenarios['ctc_treatment'] == 'keep'),
                             ['id', 'aftertax_income_percentile', 'aftertax_income_decile',
                              'aftertax_income_quintile', 'nu18', 'n24']]

In [29]:
scenario18 = pd.merge(scenario18, base_aftiq18, on='id')

In [30]:
scenario18.groupby('aftertax_income_quintile').sum()

,id,s006,keep_aftertax_income,repeal_aftertax_income,rev_neutral_ubi_aftertax_income,keep_expanded_income,repeal_expanded_income,rev_neutral_ubi_expanded_income,afti_chg_ctc,afti_chg_ubi,aftertax_income_percentile,aftertax_income_decile,nu18,n24
aftertax_income_quintile,,,,,,,,,,,,,,
0,275003034,1.976052e+05,-7.833158e+07,-7.882848e+07,-7.792633e+07,-7.393560e+07,-7.393560e+07,-7.303345e+07,84.349206,-92.426476,0,0,616.0,540.0
1,14505729587,3.378104e+07,6.879336e+08,6.803063e+08,7.044358e+08,7.313785e+08,7.313785e+08,7.555080e+08,-709.075881,inf,648428,91426,16476.0,23988.0
2,15145424438,3.397926e+07,1.840284e+09,1.804098e+09,1.843584e+09,1.984034e+09,1.984034e+09,2.023521e+09,-1240.527065,135.671302,1979073,226672,26962.0,28837.0
3,15899091087,3.397877e+07,3.127747e+09,3.065678e+09,3.123733e+09,3.498796e+09,3.498796e+09,3.556852e+09,-1377.441190,-87.615551,3489498,379587,39641.0,38613.0
4,18948105415,3.397914e+07,6.065228e+09,5.976613e+09,6.055419e+09,7.037765e+09,7.037765e+09,7.116571e+09,-1233.806749,-133.992179,5896811,626238,53810.0,49326.0
5,39406566319,3.397901e+07,4.299120e+10,4.277985e+10,4.299058e+10,5.716097e+10,5.716097e+10,5.737171e+10,-1343.057985,-148.231217,16652930,1726469,143895.0,133238.0


In [31]:
1

1